In [3]:
import geopandas as gpd
import pandas as pd
import ee
import geemap
from geowrangler import grids
import matplotlib.pyplot as plt
from shapely.geometry import mapping

In [2]:
ee.Initialize()

In [4]:
geojson_path = 'Untitled_layer.geojson'
gdf = gpd.read_file(geojson_path)

In [5]:
grid_generator = grids.SquareGridGenerator(1_000)
grid_gdf = grid_generator.generate_grid(gdf)

In [6]:
Map = geemap.Map()
Map.add_gdf(gdf, layer_name='GeoJSON Layer', style={'color': 'red', 'fillOpacity': 0})
Map.add_gdf(grid_gdf, layer_name='1km x 1km Grid', style={'color': 'black', 'fillOpacity': 0.2})
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [7]:
geojson = gdf.__geo_interface__

In [8]:
roi = ee.Geometry(geojson['features'][0]['geometry'])
modislst = ee.ImageCollection("MODIS/061/MOD11A1")\
            .select('LST_Day_1km') \
            .filterDate('2023-01-01', '2023-12-31')
start_date = '2023-11-01'
end_date = '2023-11-11'
modislst_filtered = modislst.filterDate(start_date, end_date).filterBounds(roi)

In [9]:
def extract_values(image):
    date = ee.Date(image.get('system:time_start'))
    date_str = date.format('YYYY-MM-dd')
    time_str = date.format('HH:mm:ss')
    lst = modislst_filtered.filterDate(date, date.advance(1, 'day')).mean().reduceRegion(
        reducer=ee.Reducer.mean(), geometry=roi, scale=1000).get('LST_Day_1km')
    
    return ee.Feature(None, {
        'date': date_str,
        'time': time_str,
        'modis_lst': lst
    })

features = modislst_filtered.map(extract_values).getInfo()

In [12]:
values = []
for feature in features['features']:
    date = feature['properties']['date']
    time = feature['properties']['time']
    
    lst = feature['properties'].get('modis_lst', None)
    
    values.append({
        'date': date,
        'time': time,
        'coordinates': geojson['features'][0]['geometry']['coordinates'],
        'modis_lst': lst
    })

df = pd.DataFrame(values)
df.head(10)

,date,time,coordinates,modis_lst
0,2023-11-01,00:00:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",15080.891524
1,2023-11-02,00:00:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",15108.849467
2,2023-11-03,00:00:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",15127.181029
3,2023-11-04,00:00:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",14929.135132
4,2023-11-05,00:00:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",15070.562580
5,2023-11-06,00:00:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",NaN
6,2023-11-07,00:00:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",14882.939235
7,2023-11-08,00:00:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",15080.422009
8,2023-11-09,00:00:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",15078.392114
9,2023-11-10,00:00:00,"(((87.9199927, 23.1013063), (87.9749243, 22.20...",15073.101212


In [13]:
df.to_csv('MODIS_LST_DATA.csv')